In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 400
start_epoch = 0
train_batch = 512
test_batch = 512
lr = 0.04
schedule = [75, 175, 250]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style1/aug2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.01
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

# augmentation
blur_prob = 0.1
blog_sig = 0.5
jpg_prob = 0.1

best_acc = 0

# sp
sp_alpha = 0.5
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(target_dir, 'train')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.5, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


# Loss

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [13]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [14]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [15]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

In [16]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Train

In [17]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss =  loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        arc.update(auroc, inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        if batch_idx % 100 == 0:
            print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [20]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : teacher_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 400] LR: 0.040000
1/135 | Loss:4.0366 | MainLoss:3.8431 | SPLoss:0.0000 | CLSLoss:1.9344 | top1:51.7578 | AUROC:0.5376
101/135 | Loss:1.0909 | MainLoss:0.7384 | SPLoss:0.6736 | CLSLoss:0.1574 | top1:50.5801 | AUROC:0.5100
135/135 | Loss:0.9936 | MainLoss:0.7276 | SPLoss:0.5080 | CLSLoss:0.1201 | top1:50.4314 | AUROC:0.5074
16/135 | Loss:0.6942 | MainLoss:0.6933 | SPLoss:0.0008 | CLSLoss:0.0047 | top1:50.0000 | AUROC:0.4980
63/135 | Loss:0.6931 | MainLoss:0.6922 | SPLoss:0.0008 | CLSLoss:0.0047 | top1:50.0000 | AUROC:0.6049

Epoch: [2 | 400] LR: 0.052000
1/135 | Loss:0.6974 | MainLoss:0.6966 | SPLoss:0.0008 | CLSLoss:0.0047 | top1:44.5312 | AUROC:0.4407
101/135 | Loss:0.6956 | MainLoss:0.6944 | SPLoss:0.0011 | CLSLoss:0.0069 | top1:50.4815 | AUROC:0.4994
135/135 | Loss:0.6959 | MainLoss:0.6945 | SPLoss:0.0014 | CLSLoss:0.0071 | top1:50.3396 | AUROC:0.5014
16/135 | Loss:0.6975 | MainLoss:0.6934 | SPLoss:0.0063 | CLSLoss:0.0090 | top1:49.6538 | AUROC:0.4979
63/135 | Loss:0.74

101/135 | Loss:0.7219 | MainLoss:0.6976 | SPLoss:0.0449 | CLSLoss:0.0194 | top1:49.8356 | AUROC:0.4954
135/135 | Loss:0.7198 | MainLoss:0.6975 | SPLoss:0.0408 | CLSLoss:0.0188 | top1:50.0481 | AUROC:0.4982
16/135 | Loss:0.7099 | MainLoss:0.6958 | SPLoss:0.0235 | CLSLoss:0.0234 | top1:50.4615 | AUROC:0.5104
63/135 | Loss:0.6172 | MainLoss:0.6031 | SPLoss:0.0235 | CLSLoss:0.0234 | top1:99.7414 | AUROC:1.0000

Epoch: [17 | 400] LR: 0.159938
1/135 | Loss:0.7047 | MainLoss:0.6906 | SPLoss:0.0235 | CLSLoss:0.0234 | top1:53.7109 | AUROC:0.5219
101/135 | Loss:0.7356 | MainLoss:0.6966 | SPLoss:0.0737 | CLSLoss:0.0219 | top1:50.5975 | AUROC:0.5076
135/135 | Loss:0.7460 | MainLoss:0.6965 | SPLoss:0.0946 | CLSLoss:0.0211 | top1:50.6326 | AUROC:0.5067
16/135 | Loss:0.7241 | MainLoss:0.7128 | SPLoss:0.0175 | CLSLoss:0.0251 | top1:50.3718 | AUROC:0.5114
63/135 | Loss:0.5007 | MainLoss:0.4895 | SPLoss:0.0175 | CLSLoss:0.0251 | top1:99.6511 | AUROC:1.0000

Epoch: [18 | 400] LR: 0.159911
1/135 | Loss:0.

135/135 | Loss:0.7528 | MainLoss:0.6977 | SPLoss:0.1064 | CLSLoss:0.0180 | top1:50.0802 | AUROC:0.5000
16/135 | Loss:0.7759 | MainLoss:0.7011 | SPLoss:0.1463 | CLSLoss:0.0161 | top1:50.0897 | AUROC:0.4864
63/135 | Loss:0.6456 | MainLoss:0.5708 | SPLoss:0.1463 | CLSLoss:0.0161 | top1:99.4611 | AUROC:1.0000

Epoch: [32 | 400] LR: 0.159015
1/135 | Loss:0.7780 | MainLoss:0.7032 | SPLoss:0.1463 | CLSLoss:0.0161 | top1:50.9766 | AUROC:0.4693
101/135 | Loss:0.7393 | MainLoss:0.6982 | SPLoss:0.0786 | CLSLoss:0.0189 | top1:50.0967 | AUROC:0.5046
135/135 | Loss:0.7332 | MainLoss:0.6982 | SPLoss:0.0661 | CLSLoss:0.0192 | top1:50.0117 | AUROC:0.5024
16/135 | Loss:0.6996 | MainLoss:0.6933 | SPLoss:0.0099 | CLSLoss:0.0140 | top1:50.5641 | AUROC:0.4986
63/135 | Loss:0.6223 | MainLoss:0.6160 | SPLoss:0.0099 | CLSLoss:0.0140 | top1:99.6417 | AUROC:0.9967

Epoch: [33 | 400] LR: 0.158914
1/135 | Loss:0.7006 | MainLoss:0.6942 | SPLoss:0.0099 | CLSLoss:0.0140 | top1:49.6094 | AUROC:0.5048
101/135 | Loss:0.

16/135 | Loss:0.9827 | MainLoss:0.6958 | SPLoss:0.5694 | CLSLoss:0.0213 | top1:50.0000 | AUROC:0.5032
63/135 | Loss:0.9663 | MainLoss:0.6794 | SPLoss:0.5694 | CLSLoss:0.0213 | top1:50.0000 | AUROC:0.9877

Epoch: [47 | 400] LR: 0.156996
1/135 | Loss:0.9770 | MainLoss:0.6902 | SPLoss:0.5694 | CLSLoss:0.0213 | top1:53.9062 | AUROC:0.4949
101/135 | Loss:1.5229 | MainLoss:0.6925 | SPLoss:1.6570 | CLSLoss:0.0185 | top1:52.1948 | AUROC:0.5372
135/135 | Loss:1.4407 | MainLoss:0.6929 | SPLoss:1.4920 | CLSLoss:0.0185 | top1:51.9604 | AUROC:0.5328
16/135 | Loss:1.7773 | MainLoss:0.6862 | SPLoss:2.1782 | CLSLoss:0.0195 | top1:56.7949 | AUROC:0.5972
63/135 | Loss:1.7001 | MainLoss:0.6090 | SPLoss:2.1782 | CLSLoss:0.0195 | top1:97.6729 | AUROC:0.9999

Epoch: [48 | 400] LR: 0.156823
1/135 | Loss:1.7802 | MainLoss:0.6891 | SPLoss:2.1782 | CLSLoss:0.0195 | top1:53.5156 | AUROC:0.5570
101/135 | Loss:1.1864 | MainLoss:0.6959 | SPLoss:0.9773 | CLSLoss:0.0174 | top1:51.0810 | AUROC:0.5130
135/135 | Loss:1.

63/135 | Loss:0.9005 | MainLoss:0.7007 | SPLoss:0.3970 | CLSLoss:0.0131 | top1:43.6106 | AUROC:0.3091

Epoch: [62 | 400] LR: 0.153910
1/135 | Loss:0.8923 | MainLoss:0.6925 | SPLoss:0.3970 | CLSLoss:0.0131 | top1:51.9531 | AUROC:0.5107
101/135 | Loss:1.0072 | MainLoss:0.6937 | SPLoss:0.6239 | CLSLoss:0.0155 | top1:51.0094 | AUROC:0.5156
135/135 | Loss:1.0112 | MainLoss:0.6938 | SPLoss:0.6317 | CLSLoss:0.0150 | top1:50.9183 | AUROC:0.5133
16/135 | Loss:1.0751 | MainLoss:0.6926 | SPLoss:0.7632 | CLSLoss:0.0093 | top1:50.9231 | AUROC:0.5151
63/135 | Loss:1.0118 | MainLoss:0.6293 | SPLoss:0.7632 | CLSLoss:0.0093 | top1:97.3770 | AUROC:0.9981

Epoch: [63 | 400] LR: 0.153668
1/135 | Loss:1.0735 | MainLoss:0.6910 | SPLoss:0.7632 | CLSLoss:0.0093 | top1:51.3672 | AUROC:0.5296
101/135 | Loss:1.1926 | MainLoss:0.6907 | SPLoss:1.0007 | CLSLoss:0.0156 | top1:52.4733 | AUROC:0.5299
135/135 | Loss:1.2765 | MainLoss:0.6912 | SPLoss:1.1674 | CLSLoss:0.0165 | top1:52.3773 | AUROC:0.5343
16/135 | Loss:1.

...

1/135 | Loss:0.5419 | MainLoss:0.3375 | SPLoss:0.3769 | CLSLoss:0.1592 | top1:86.3281 | AUROC:0.9314
101/135 | Loss:0.5580 | MainLoss:0.3538 | SPLoss:0.3766 | CLSLoss:0.1593 | top1:84.9319 | AUROC:0.9237
135/135 | Loss:0.5567 | MainLoss:0.3540 | SPLoss:0.3733 | CLSLoss:0.1597 | top1:85.0236 | AUROC:0.9235
16/135 | Loss:0.4979 | MainLoss:0.3039 | SPLoss:0.3557 | CLSLoss:0.1609 | top1:87.8846 | AUROC:0.9465
63/135 | Loss:1.1937 | MainLoss:0.9997 | SPLoss:0.3557 | CLSLoss:0.1609 | top1:55.0467 | AUROC:0.6760

Epoch: [183 | 400] LR: 0.000981
1/135 | Loss:0.5044 | MainLoss:0.3104 | SPLoss:0.3557 | CLSLoss:0.1609 | top1:87.5000 | AUROC:0.9428
101/135 | Loss:0.5514 | MainLoss:0.3540 | SPLoss:0.3618 | CLSLoss:0.1641 | top1:84.9281 | AUROC:0.9234
135/135 | Loss:0.5536 | MainLoss:0.3525 | SPLoss:0.3693 | CLSLoss:0.1638 | top1:85.0207 | AUROC:0.9241
16/135 | Loss:0.5113 | MainLoss:0.3031 | SPLoss:0.3835 | CLSLoss:0.1643 | top1:87.2949 | AUROC:0.9465
63/135 | Loss:1.2717 | MainLoss:1.0635 | SPLoss

101/135 | Loss:0.5470 | MainLoss:0.3270 | SPLoss:0.4051 | CLSLoss:0.1752 | top1:86.4113 | AUROC:0.9355
135/135 | Loss:0.5440 | MainLoss:0.3259 | SPLoss:0.4011 | CLSLoss:0.1755 | top1:86.4768 | AUROC:0.9360
16/135 | Loss:0.5148 | MainLoss:0.3039 | SPLoss:0.3866 | CLSLoss:0.1757 | top1:87.3462 | AUROC:0.9499
63/135 | Loss:1.0527 | MainLoss:0.8418 | SPLoss:0.3866 | CLSLoss:0.1757 | top1:62.3302 | AUROC:0.7917

Epoch: [198 | 400] LR: 0.000888
1/135 | Loss:0.5349 | MainLoss:0.3241 | SPLoss:0.3866 | CLSLoss:0.1757 | top1:85.5469 | AUROC:0.9457
101/135 | Loss:0.5166 | MainLoss:0.3157 | SPLoss:0.3663 | CLSLoss:0.1778 | top1:86.9373 | AUROC:0.9401
135/135 | Loss:0.5175 | MainLoss:0.3168 | SPLoss:0.3658 | CLSLoss:0.1778 | top1:86.9141 | AUROC:0.9398
16/135 | Loss:0.4922 | MainLoss:0.2921 | SPLoss:0.3645 | CLSLoss:0.1779 | top1:88.1026 | AUROC:0.9490
63/135 | Loss:1.2721 | MainLoss:1.0720 | SPLoss:0.3645 | CLSLoss:0.1779 | top1:55.4361 | AUROC:0.6507

Epoch: [199 | 400] LR: 0.000882
1/135 | Loss:

135/135 | Loss:0.5434 | MainLoss:0.3150 | SPLoss:0.4199 | CLSLoss:0.1846 | top1:86.8835 | AUROC:0.9412
16/135 | Loss:0.4952 | MainLoss:0.2763 | SPLoss:0.4008 | CLSLoss:0.1842 | top1:88.7564 | AUROC:0.9575
63/135 | Loss:1.2725 | MainLoss:1.0537 | SPLoss:0.4008 | CLSLoss:0.1842 | top1:54.8100 | AUROC:0.6384

Epoch: [213 | 400] LR: 0.000794
1/135 | Loss:0.5001 | MainLoss:0.2813 | SPLoss:0.4008 | CLSLoss:0.1842 | top1:87.3047 | AUROC:0.9576
101/135 | Loss:0.5089 | MainLoss:0.3029 | SPLoss:0.3747 | CLSLoss:0.1866 | top1:87.5851 | AUROC:0.9448
135/135 | Loss:0.5195 | MainLoss:0.3022 | SPLoss:0.3972 | CLSLoss:0.1873 | top1:87.6064 | AUROC:0.9455
16/135 | Loss:0.5256 | MainLoss:0.2695 | SPLoss:0.4742 | CLSLoss:0.1904 | top1:89.2051 | AUROC:0.9583
63/135 | Loss:1.4062 | MainLoss:1.1501 | SPLoss:0.4742 | CLSLoss:0.1904 | top1:53.6760 | AUROC:0.6022

Epoch: [214 | 400] LR: 0.000787
1/135 | Loss:0.5613 | MainLoss:0.3051 | SPLoss:0.4742 | CLSLoss:0.1904 | top1:88.2812 | AUROC:0.9465
101/135 | Loss:

16/135 | Loss:3.0010 | MainLoss:0.2683 | SPLoss:5.4269 | CLSLoss:0.1924 | top1:89.0000 | AUROC:0.9587
63/135 | Loss:3.9525 | MainLoss:1.2198 | SPLoss:5.4269 | CLSLoss:0.1924 | top1:50.9533 | AUROC:0.5134

Epoch: [228 | 400] LR: 0.000700
1/135 | Loss:2.9927 | MainLoss:0.2600 | SPLoss:5.4269 | CLSLoss:0.1924 | top1:90.8203 | AUROC:0.9621
101/135 | Loss:4.0033 | MainLoss:0.2997 | SPLoss:7.3687 | CLSLoss:0.1924 | top1:87.7146 | AUROC:0.9465
135/135 | Loss:3.7059 | MainLoss:0.3000 | SPLoss:6.7734 | CLSLoss:0.1927 | top1:87.6734 | AUROC:0.9467
16/135 | Loss:2.4786 | MainLoss:0.2589 | SPLoss:4.4007 | CLSLoss:0.1935 | top1:89.6410 | AUROC:0.9621
63/135 | Loss:3.4557 | MainLoss:1.2360 | SPLoss:4.4007 | CLSLoss:0.1935 | top1:51.4891 | AUROC:0.5517

Epoch: [229 | 400] LR: 0.000694
1/135 | Loss:2.5246 | MainLoss:0.3049 | SPLoss:4.4007 | CLSLoss:0.1935 | top1:87.1094 | AUROC:0.9504
101/135 | Loss:1.9363 | MainLoss:0.2986 | SPLoss:3.2364 | CLSLoss:0.1943 | top1:87.7630 | AUROC:0.9467
135/135 | Loss:

KeyboardInterrupt: 